1) 用 Word2vec 模型（自己训练的和rusvectores的）向量化文本，根据重述定义在所得向量上训练2个模型，找到最好的模型。

Обучите 2 модели по определению перефразирования на получившихся векторах
Word2Vec要在另外的库训练（不在平行句子上），可以选用课堂上的数据或其它的

要用cross-validation评价模型，кросс-валидации! Метрика - f1

2) 分别用5种方法训练成对的文本向量：SVD, NMF, Word2Vec (自己的和rucvectores的), Fastext. 
数据库的每行应有五对向量。在每对向量之间计算余弦相似度（一对5个） 

通过这些相似度建立训练样本。在此样本上训练任何一个模型（Logreg，Random Forest或其他），
并评估cross-validation(交叉验证)的质量（使用micro-f1-measure）。
尝试通过更改向量化时的参数来改进指标。

SVD 和 NMF 直接应用于数据,  w2w 和 fastext 在其他库训练（与第一问一样） 

Выложите код к себе на гитхаб и вставьте ссылку в поле ниже (в тетрадке должны быть показатели метрик и ваши комментарии).


In [2]:
import pandas as pd
from lxml import html
import numpy as np
from tqdm import tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import TruncatedSVD, NMF, PCA
from sklearn.manifold import TSNE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_distances
from sklearn.ensemble import RandomForestClassifier
import gensim
from sklearn.cluster import MiniBatchKMeans
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from collections import Counter,defaultdict
from string import punctuation
import os
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
%matplotlib inline

morph = MorphAnalyzer()
punct = punctuation+'«»—…“”*№–'
stops = set(stopwords.words('russian'))

In [3]:
def normalize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]
    words = [morph.parse(word)[0].normal_form for word in words if word and word not in stops]

    return ' '.join(words)

def tokenize(text):
    
    words = [word.strip(punct) for word in text.lower().split()]

    return ' '.join(words)

## corpus_hum

In [5]:
data = open('corpus_hum.txt', encoding="utf-8").read().splitlines()
data_norm = [normalize(text) for text in data]
data_norm = [text for text in data_norm if text]

In [83]:
data_norm[:2]

['абай василий васо иван 1900–2001 русский лингвист родиться 2 15 декабрь 1900 с.коби тифлисский губерния ныне грузия 1925 окончить факультет общественный наука ленинградский университет 1928 аспирантура 1928–1930 сотрудник кавказский историко-археологический институт ан ссср 1930 полвека работать яфетический институт затем институт язык мышление институт языкознание ан ссср ленинград 1950 москва доктор филологический наука 1962 профессор 1969 лауреат государственный премия ссср 1981 почётный член азиатский королевский общество великобритания ирландия 1966 член-корреспондент финно-угорский общество хельсинки 1973 умереть абай москва 18 март 2001',
 'также тема']

In [7]:
len(data_norm)

115082

In [8]:
cv = CountVectorizer(min_df=3, max_df=0.4, max_features=1000)
X = cv.fit_transform(data_norm)

In [9]:
X.shape

(115082, 1000)

# w2v

## w2v самостоятельно обученная (лучше)

In [31]:
corpus_xml = html.fromstring(open('paraphrases.xml', 'rb').read())
texts_1 = []
texts_2 = []
classes = []

for p in corpus_xml.xpath('//paraphrase'):
    texts_1.append(p.xpath('./value[@name="text_1"]/text()')[0])
    texts_2.append(p.xpath('./value[@name="text_2"]/text()')[0])
    classes.append(p.xpath('./value[@name="class"]/text()')[0])
    
data = pd.DataFrame({'text_1':texts_1, 'text_2':texts_2, 'label':classes})

In [32]:
data['text_1_norm'] = data['text_1'].apply(normalize)
data['text_2_norm'] = data['text_2'].apply(normalize)

In [61]:
data.head(10)

,text_1,text_2,label,text_1_norm,text_2_norm
0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейский разрешить стрелять поражение гражд...,полиция мочь разрешить стрелять хулиган травма...
1,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,право полицейский проникновение жилища решить ...,правило внесудебный проникновение полицейский ...
2,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,президент египет ввести чрезвычайный положение...,власть египет угрожать ввести страна чрезвычай...
3,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,вернуться сирия россиянин волновать вопрос тру...,самолёт мчс вывезти россиянин разрушить сирия
4,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,москва сирия вернуться 2 самолёт мчс россиянин...,самолёт мчс вывезти россиянин разрушить сирия
5,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,1,пристав собрать отпечаток палец российский дол...,пристав снять отпечаток палец злостный неплате...
6,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,-1,саратовский дебошир борт самолёт москва хургад...,саратовский дебошир отказываться возвращаться ...
7,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,0,цик хотеть отказаться электронный система подс...,цик отказаться электронный средство подсчёт голос
8,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,-1,суд петербург оставить дело гибель подросток п...,лондонский гайд-парк это место митинг прежде парк
9,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,-1,страна опека сократить добыча нефть 1 миллион ...,обама продлить полномочие наса сотрудничество ...


In [169]:
w2v = gensim.models.Word2Vec([text.split() for text in data_norm], size=300, sg=1)

In [73]:
def get_embedding(text, model, dim):
    text = text.split()
    
    # чтобы не доставать одно слово несколько раз
    # сделаем счетчик, а потом векторы домножим на частоту
    words = Counter(text)
    total = len(text)
    vectors = np.zeros((len(words), dim))
    
    for i,word in enumerate(words):
        try:
            v = model[word]
            vectors[i] = v*(words[word]/total) # просто умножаем вектор на частоту
        except (KeyError, ValueError):
            continue
    
    if vectors.any():
        vector = np.average(vectors, axis=0)
    else:
        vector = np.zeros((dim))
    
    return vector

In [170]:
dim = 300
X_text_1_w2v = np.zeros((len(data['text_1_norm']), dim))
X_text_2_w2v = np.zeros((len(data['text_2_norm']), dim))

for i, text in enumerate(data['text_1_norm'].values):
    X_text_1_w2v[i] = get_embedding(text, w2v, dim)
    
for i, text in enumerate(data['text_2_norm'].values):
    X_text_2_w2v[i] = get_embedding(text, w2v, dim)

d:\program files\python\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [171]:
X_text_w2v = np.concatenate([X_text_1_w2v, X_text_2_w2v], axis=1)

In [172]:
X_text_w2v.shape

(7227, 600)

### cross valid

In [86]:
from sklearn.model_selection import cross_val_score

In [37]:
y = data['label'].values
print(y.shape)

(7227,)


In [173]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_w2v, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.48      0.50      0.49       629
           0       0.46      0.57      0.51       737
           1       0.38      0.21      0.27       441

    accuracy                           0.46      1807
   macro avg       0.44      0.43      0.43      1807
weighted avg       0.45      0.46      0.45      1807



In [174]:
cross_val_score(clf, X_text_w2v, y, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.4445622381646162

In [176]:
from sklearn.neighbors import KNeighborsClassifier
k_range = range(1,31)
cv_scores = []
for n in k_range:
    knn = KNeighborsClassifier(n)
    scores = cross_val_score(knn, X_text_w2v, y, cv=10,scoring='f1_micro')
    cv_scores.append(scores.mean())

KeyboardInterrupt: 

In [ ]:
from matplotlib import pyplot as plt
plt.plot(k_range,cv_scores)
plt.xlabel('K')
plt.ylabel('f1_micro')
plt.show()

In [ ]:
best_knn = KNeighborsClassifier(n_neighbors=20)
best_knn.fit(train_X,train_y)
print(best_knn.score(valid_X,valid_y))

## w2v rusvectores

In [105]:
data_tag=pd.read_csv('data_paraphraser_norm.csv')

In [111]:
data_tag.head(10)

,label,text_1,text_2,text_1_norm,text_2_norm
0,0,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,полицейский_NOUN разрешать_VERB стрелять_VERB ...,полиция_NOUN мочь_VERB разрешать_VERB стрелять...
1,0,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,право_ADV полицейский_NOUN на_ADP проникновени...,правило_NOUN внесудебный_ADJ проникновение_NOU...
2,0,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,президент_NOUN египет_NOUN вводить_VERB чрезвы...,власть_NOUN египет_NOUN угрожать_VERB вводить_...
3,-1,Вернувшихся из Сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,вернуться_VERB из_ADP сирия_NOUN россиянин_NOU...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...
4,0,В Москву из Сирии вернулись 2 самолета МЧС с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,в_ADP москва_NOUN из_ADP сирия_NOUN вернуться_...,самолет_NOUN мчс_NOUN вывозить_VERB россиянин_...
5,1,Приставы соберут отпечатки пальцев российских ...,Приставы снимут отпечатки пальцев у злостных н...,пристав_NOUN собирать_VERB отпечаток_NOUN пале...,пристав_NOUN снимать_VERB отпечаток_NOUN палец...
6,-1,На саратовского дебошира с борта самолета Моск...,Саратовский дебошир отказывается возвращаться ...,на_ADP саратовский_ADJ дебошир_NOUN с_ADP борт...,саратовский_ADJ дебошир_NOUN отказываться_VERB...
7,0,ЦИК хочет отказаться от электронной системы по...,ЦИК может отказаться от электронных средств по...,цик_NOUN хотеть_VERB отказываться_VERB от_ADP ...,цик_NOUN мочь_VERB отказываться_VERB от_ADP эл...
8,-1,Суд Петербурга оставил на потом дело о гибели ...,Лондонский Гайд-парк - это не место для митинг...,суд_NOUN петербург_NOUN оставлять_VERB на_ADP ...,лондонский_ADJ гайд_NOUN парк_NOUN это_PART не...
9,-1,Страны ОПЕК сократили добычу нефти на 1 млн ба...,Обама продлил полномочия НАСА по сотрудничеств...,страна_NOUN опек_NOUN сокращать_VERB добыча_NO...,обама_NOUN продлять_VERB полномочие_NOUN нас_N...


In [112]:
data_tag['text_1_norm'][0]

'полицейский_NOUN разрешать_VERB стрелять_VERB на_ADP поражение_NOUN по_ADP гражданин_NOUN с_ADP травматика_NOUN'

In [113]:
y_tag = data_tag['label'].values

In [155]:
rv = gensim.models.KeyedVectors.load_word2vec_format('model.bin', binary=True)

In [156]:
rv.most_similar('полицейский_NOUN')

[('милиционер_NOUN', 0.7502387762069702),
 ('полицейский_ADJ', 0.7121201157569885),
 ('полиция_NOUN', 0.7049249410629272),
 ('стражник_NOUN', 0.6457393169403076),
 ('полисмена_NOUN', 0.6368731260299683),
 ('жандарм_NOUN', 0.6188533306121826),
 ('охранник_NOUN', 0.6150203943252563),
 ('бандит_NOUN', 0.6047532558441162),
 ('патрульный_NOUN', 0.6029660105705261),
 ('сыщик_NOUN', 0.601699709892273)]

In [159]:
len(rv['полицейский_NOUN'])

300

In [160]:
dim = 300
X_text_1_rv = np.zeros((len(data_tag['text_1_norm']), dim))
X_text_2_rv = np.zeros((len(data_tag['text_2_norm']), dim))

for i, text in enumerate(data_tag['text_1_norm'].values):
    X_text_1_rv[i] = get_embedding(text, rv, dim)
    
for i, text in enumerate(data_tag['text_2_norm'].values):
    X_text_2_rv[i] = get_embedding(text, rv, dim)

In [161]:
X_text_rv = np.concatenate([X_text_1_rv, X_text_2_rv], axis=1)

In [162]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_rv, y_tag,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.48      0.51      0.49       629
           0       0.45      0.54      0.49       737
           1       0.34      0.19      0.25       441

    accuracy                           0.45      1807
   macro avg       0.42      0.42      0.41      1807
weighted avg       0.43      0.45      0.43      1807



In [163]:
cross_val_score(clf, X_text_rv, y_tag, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.4190825780320484

# 2 часть

tfidf

In [145]:
tfidf = TfidfVectorizer(min_df=3, max_df=0.4, max_features=1000)
tfidf.fit(pd.concat([data['text_1_norm'], data['text_2_norm']]))

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=0.4, max_features=1000,
                min_df=3, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

## SVD

In [177]:
svd = TruncatedSVD(300)

X_text_1_svd = svd.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_svd = svd.fit_transform(tfidf.transform(data['text_2_norm']))

X_text_svd = np.concatenate([X_text_1_svd, X_text_2_svd], axis=1)

In [178]:
X_text_svd.shape

(7227, 600)

### cos similarity

In [51]:
from sklearn.metrics.pairwise import cosine_similarity

In [224]:
cos_sim_svd = [cosine_similarity(v.reshape(1, -1), X_text_2_svd[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_svd)]
cos_sim_svd = [str(x) for x in cos_sim_svd]
cos_sim_svd = np.array(cos_sim_svd, dtype=np.object)

In [225]:
cos_sim_svd[:10]

array(['-0.019408663139645774', '0.010566483155286561',
       '0.05013614716396606', '-0.1115259826532875', '-0.119719620440669',
       '0.00773667661061782', '-0.0052200624284161855',
       '0.05696724303781667', '0.03071753115177813',
       '0.03872764745398728'], dtype=object)

In [221]:
y[:10]

array(['0', '0', '0', '-1', '0', '1', '-1', '0', '-1', '-1'], dtype=object)

In [218]:
cos_dis_svd = [cosine_distances(v.reshape(1, -1), X_text_2_svd[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_svd)]

In [219]:
cos_dis_svd[:10]

[1.0194086631396457,
 0.9894335168447135,
 0.9498638528360339,
 1.1115259826532875,
 1.119719620440669,
 0.9922633233893822,
 1.0052200624284162,
 0.9430327569621834,
 0.9692824688482219,
 0.9612723525460127]

### cross valid

In [ ]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_svd, cos_sim_svd,random_state=1)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

In [184]:
cross_val_score(X_text_svd, cos_sim_svd, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.4154964139852141

## NMF

In [195]:
nmf = NMF(n_components=300, alpha=0.01)

X_text_1_nmf = nmf.fit_transform(tfidf.transform(data['text_1_norm']))
X_text_2_nmf = nmf.fit_transform(tfidf.transform(data['text_2_norm']))

X_text_nmf = np.concatenate([X_text_1_nmf, X_text_2_nmf], axis=1)

### cos similarity

In [196]:
cos_sim_nmf = [cosine_similarity(v.reshape(1, -1), X_text_2_nmf[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_nmf)]

In [197]:
cos_sim_nmf[:10]

[0.0002698119141306164,
 0.0,
 0.0,
 1.3074990882437725e-05,
 0.24781095598258893,
 0.0,
 0.0,
 0.006705448943948389,
 0.001106753692925767,
 0.008193160517522096]

In [198]:
cos_dis_nmf = [cosine_distances(v.reshape(1, -1), X_text_2_nmf[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_nmf)]
cos_dis_nmf[:10]

[0.9997301880858693,
 1.0,
 1.0,
 0.9999869250091176,
 0.752189044017411,
 1.0,
 1.0,
 0.9932945510560516,
 0.9988932463070742,
 0.9918068394824779]

### cross valid

In [199]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_nmf, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.48      0.51      0.50       629
           0       0.46      0.53      0.49       737
           1       0.35      0.24      0.29       441

    accuracy                           0.45      1807
   macro avg       0.43      0.43      0.43      1807
weighted avg       0.44      0.45      0.45      1807



In [200]:
cross_val_score(clf, X_text_nmf, y, scoring='f1_micro', cv=10).mean()

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver 

0.4265578017323497

## W2V самостоятельно обученная

In [153]:
cos_sim_w2v = [cosine_similarity(v.reshape(1, -1), X_text_2_w2v[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_w2v)]
cos_sim_w2v[:10]

[0.9093553040093278,
 0.9162935532635855,
 0.9508713657789876,
 0.7610319787133144,
 0.9083803325663123,
 0.9244763677726099,
 0.691226839678667,
 0.9688288204766886,
 0.7165422628726466,
 0.7288469007395649]

In [142]:
cos_dis_w2v = [cosine_distances(v.reshape(1, -1), X_text_2_w2v[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_w2v)]
cos_dis_w2v[:10]

[0.09064469599067215,
 0.08370644673641447,
 0.049128634221012435,
 0.2389680212866856,
 0.09161966743368766,
 0.07552363222739011,
 0.30877316032133295,
 0.031171179523311388,
 0.28345773712735345,
 0.27115309926043507]

## W2V rusvectores

In [164]:
cos_sim_rv = [cosine_similarity(v.reshape(1, -1), X_text_2_rv[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_rv)]
cos_sim_rv[:10]

[0.7714457202726551,
 0.779862242937686,
 0.8482040968389215,
 0.6386970316706366,
 0.7892051301966663,
 0.835266357833032,
 0.6854796663413958,
 0.9471294060292673,
 0.43523004109016716,
 0.43741336716064017]

In [165]:
cos_dis_rv = [cosine_distances(v.reshape(1, -1), X_text_2_rv[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_rv)]
cos_dis_rv[:10]

[0.22855427972734488,
 0.22013775706231398,
 0.1517959031610785,
 0.36130296832936337,
 0.21079486980333373,
 0.16473364216696795,
 0.3145203336586042,
 0.05287059397073268,
 0.5647699589098328,
 0.5625866328393598]

## FastText

In [135]:
fast_text = gensim.models.FastText([text.split() for text in data_norm], size=50, min_n=4, max_n=8)

In [136]:
dim = 50
data['text_1_notnorm'] = data['text_1'].apply(tokenize)
data['text_2_notnorm'] = data['text_2'].apply(tokenize)

X_text_1_ft = np.zeros((len(data['text_1_notnorm']), dim))
X_text_2_ft = np.zeros((len(data['text_2_notnorm']), dim))

for i, text in enumerate(data['text_1_notnorm'].values):
    X_text_1_ft[i] = get_embedding(text, fast_text, dim)
    
for i, text in enumerate(data['text_2_notnorm'].values):
    X_text_2_ft[i] = get_embedding(text, fast_text, dim)

d:\program files\python\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


In [138]:
cos_sim_ft = [cosine_similarity(v.reshape(1, -1), X_text_2_ft[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_ft)]
cos_sim_ft[:10]

[0.8030769453455194,
 0.7447322572955146,
 0.6308851035754462,
 0.3842010740534307,
 0.265047370047229,
 0.7304068289115696,
 0.4336536261772452,
 0.8857981894785179,
 0.2316844764635385,
 0.10900673113157934]

In [144]:
cos_dis_ft = [cosine_distances(v.reshape(1, -1), X_text_2_ft[i].reshape(1, -1)).tolist()[0][0] for i, v in enumerate(X_text_1_ft)]
cos_dis_ft[:10]

[0.19692305465448057,
 0.2552677427044854,
 0.36911489642455375,
 0.6157989259465693,
 0.734952629952771,
 0.2695931710884304,
 0.5663463738227548,
 0.11420181052148215,
 0.7683155235364615,
 0.8909932688684207]

### cross valid

In [193]:
train_X, valid_X, train_y, valid_y = train_test_split(X_text_ft, y,random_state=1)
clf = LogisticRegression(C=1000)
clf.fit(train_X, train_y)
preds = clf.predict(valid_X)
print(classification_report(valid_y, preds))

d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
d:\program files\python\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

          -1       0.49      0.54      0.51       629
           0       0.46      0.54      0.49       737
           1       0.39      0.22      0.28       441

    accuracy                           0.46      1807
   macro avg       0.44      0.43      0.43      1807
weighted avg       0.45      0.46      0.45      1807



In [ ]:
cross_val_score(clf, X_text_ft, y, scoring='f1_micro', cv=10).mean()